In [24]:
import os
import time
import zipfile
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
)
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from itertools import product
from sklearn.decomposition import PCA


###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList


def flastVectorization(dataPoints, dim=0, eps=0.3):
    """Performs vectorization using CountVectorizer with optional dimensionality reduction via PCA."""
    countVec = CountVectorizer(stop_words=None)  # No stop word removal
    Z_full = countVec.fit_transform(dataPoints).toarray()  # Convert to dense matrix for PCA

    if dim <= 0:
        # Automatically set the number of components based on the variance explained
        pca = PCA(n_components=eps)  # Use 'eps' as the variance to explain
        Z = pca.fit_transform(Z_full)
    else:
        # Use specified number of components
        pca = PCA(n_components=dim)
        Z = pca.fit_transform(Z_full)

    return Z


###############################################################################
# Modified KNN with Threshold Adjustment

def flastKNNWithThresholds(outDir, flakyZip, nonFlakyZip, extractDir, n_splits, dim, eps, combination_label, param_grid):
    v0 = time.perf_counter()

    # Extract the zip files
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    os.makedirs(flakyDir, exist_ok=True)
    os.makedirs(nonFlakyDir, exist_ok=True)
    
    extract_zip(flakyZip, flakyDir)
    extract_zip(nonFlakyZip, nonFlakyDir)

    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky

    print(f"Number of flaky documents: {len(dataPointsFlaky)}")
    print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
    print(f"Total number of documents: {len(dataPoints)}")
    
    if len(dataPoints) == 0:
        raise ValueError("No documents available for vectorization. Please check the input directories.")

    # Vectorization
    Z = flastVectorization(dataPoints, dim=dim, eps=eps)
    dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))
    vecTime = time.perf_counter() - v0

    # Define StratifiedKFold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Initialize storage for metrics per fold and threshold
    thresholds = np.linspace(0.1, 0.9, 9)
    metrics_per_combination = []

    # Manually iterate over all combinations of hyperparameters
    for params in product(*param_grid.values()):
        # Convert params from tuple to dictionary
        param_dict = dict(zip(param_grid.keys(), params))
        print(f"Training with parameters: {param_dict}")
        
        # Initialize KNeighborsClassifier with current hyperparameter combination
        knn_model = KNeighborsClassifier(
            n_neighbors=param_dict['n_neighbors'],
            weights=param_dict['weights'],
            metric=param_dict['metric'],
            n_jobs=-1
        )
        
        # Cross-validation
        for fold, (train_index, test_index) in enumerate(skf.split(Z, dataLabelsList)):
            X_train, X_test = Z[train_index], Z[test_index]
            y_train, y_test = dataLabelsList[train_index], dataLabelsList[test_index]

            if sum(y_train) == 0 or sum(y_test) == 0:
                print(f"Skipping fold {fold+1} due to no positive samples in train or test set")
                continue

            # Train the model
            knn_model.fit(X_train, y_train)

            # Predict probabilities on test set
            if hasattr(knn_model, "predict_proba"):
                y_pred_proba = knn_model.predict_proba(X_test)
            else:
                # If predict_proba is not available, use distance-based probabilities
                distances, indices = knn_model.kneighbors(X_test)
                weights = knn_model._get_weights(distances)
                y_pred_proba = np.zeros((X_test.shape[0], 2))
                for i, neighbors in enumerate(indices):
                    neighbor_labels = y_train[neighbors]
                    if weights is None:
                        proba = np.bincount(neighbor_labels, minlength=2) / knn_model.n_neighbors
                    else:
                        proba = np.bincount(neighbor_labels, weights=weights[i], minlength=2) / weights[i].sum()
                    y_pred_proba[i] = proba

            # Calculate metrics for each threshold
            for threshold in thresholds:
                y_pred = (y_pred_proba[:, 1] >= threshold).astype(int)

                # Calculate metrics for this threshold
                f1 = f1_score(y_test, y_pred, zero_division=1)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred, zero_division=1)
                recall = recall_score(y_test, y_pred, zero_division=1)
                mcc = matthews_corrcoef(y_test, y_pred)

                metrics_per_combination.append({
                    'n_neighbors': param_dict['n_neighbors'],
                    'weights': param_dict['weights'],
                    'metric': param_dict['metric'],
                    'fold': fold + 1,
                    'threshold': threshold,
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'mcc': mcc
                })

    if len(metrics_per_combination) == 0:
        print("No valid folds. Exiting.")
        return None, None

    # Convert the list of metrics into a DataFrame
    df_results = pd.DataFrame(metrics_per_combination)

    # Save the per-fold results
    outFile = f"{combination_label}-thresholds-knn-results-per-fold.csv"
    df_results.to_csv(os.path.join(outDir, outFile), index=False)

    print(f"Per-fold KNN analysis completed. Results saved to: {outFile}")

    # Compute the average metrics across folds for each combination of hyperparameters and thresholds
    grouped_metrics = df_results.groupby(
        ['n_neighbors', 'weights', 'metric', 'threshold']
    ).agg({
        'accuracy': ['mean', 'std'],
        'precision': ['mean', 'std'],
        'recall': ['mean', 'std'],
        'f1': ['mean', 'std'],
        'mcc': ['mean', 'std']
    }).reset_index()

    # Flatten the MultiIndex columns
    grouped_metrics.columns = [
        '_'.join(col).strip('_') if col[1] else col[0] 
        for col in grouped_metrics.columns.values
    ]

    # Save the averaged results
    outFileAvg = f"{combination_label}-thresholds-knn-results-averaged.csv"
    grouped_metrics.to_csv(os.path.join(outDir, outFileAvg), index=False)

    print(f"Averaged results saved to: {outFileAvg}")

    return df_results, grouped_metrics

if __name__ == "__main__":

    # Define the parameter grid for hyperparameter tuning
    param_grid = {
        'n_neighbors': [3, 5, 7, 9, 11, 15, 20],
        'weights': ['uniform', 'distance'],
        'metric': ['cosine', 'euclidean'],
    }

    # Paths to your datasets
    flakyZip = "compressedDataset/flaky_files.zip"
    # Use the same dataset for both models to ensure fairness
    # You can choose between 'reduced_nonflaky_files.zip' (balanced) or 'all_nonflaky_files.zip' (unbalanced)
    nonFlakyZip = "compressedDataset/all_nonflaky_files.zip"  # Unbalanced dataset
    # nonFlakyZip = "compressedDataset/reduced_nonflaky_files.zip"  # Balanced dataset

    # Create result and extract directories
    outDir = "results/knn_thresholds/"
    os.makedirs(outDir, exist_ok=True)
    extractDir = "extracted/knn_thresholds/"
    os.makedirs(extractDir, exist_ok=True)

    # Perform KNN analysis with threshold adjustments
    print("Starting KNN analysis with threshold adjustments...")
    df_results, grouped_metrics = flastKNNWithThresholds(
        outDir, flakyZip, nonFlakyZip, extractDir, n_splits=5, dim=100, eps=0.3,
        combination_label="knn_thresholds", param_grid=param_grid)

    if df_results is not None:
        print("Analysis completed. Per-fold Results:")
        print(df_results.head())

        print("Averaged Results:")
        print(grouped_metrics.head())
    else:
        print("Analysis did not produce any results.")


Starting KNN analysis with threshold adjustments...
Number of flaky documents: 45
Number of non-flaky documents: 254
Total number of documents: 299
Training with parameters: {'n_neighbors': 3, 'weights': 'uniform', 'metric': 'cosine'}
Training with parameters: {'n_neighbors': 3, 'weights': 'uniform', 'metric': 'euclidean'}
Training with parameters: {'n_neighbors': 3, 'weights': 'distance', 'metric': 'cosine'}
Training with parameters: {'n_neighbors': 3, 'weights': 'distance', 'metric': 'euclidean'}
Training with parameters: {'n_neighbors': 5, 'weights': 'uniform', 'metric': 'cosine'}
Training with parameters: {'n_neighbors': 5, 'weights': 'uniform', 'metric': 'euclidean'}
Training with parameters: {'n_neighbors': 5, 'weights': 'distance', 'metric': 'cosine'}
Training with parameters: {'n_neighbors': 5, 'weights': 'distance', 'metric': 'euclidean'}
Training with parameters: {'n_neighbors': 7, 'weights': 'uniform', 'metric': 'cosine'}
Training with parameters: {'n_neighbors': 7, 'weights

In [22]:
import os
import time
import zipfile
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
)
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from itertools import product


###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList


def flastVectorization(dataPoints, dim=0, eps=0.3):
    """Performs vectorization using CountVectorizer with optional dimensionality reduction via PCA."""
    countVec = CountVectorizer(stop_words=None)  # No stop word removal
    Z_full = countVec.fit_transform(dataPoints).toarray()  # Convert to dense matrix for PCA

    if dim <= 0:
        # Automatically set the number of components based on the variance explained
        pca = PCA(n_components=eps)  # Use 'eps' as the variance to explain
        Z = pca.fit_transform(Z_full)
    else:
        # Use specified number of components
        pca = PCA(n_components=dim)
        Z = pca.fit_transform(Z_full)

    return Z


###############################################################################
# Modified KNN with Threshold Adjustment

def flastKNNWithThresholds(outDir, flakyZip, nonFlakyZip, extractDir, n_splits, dim, eps, combination_label, param_grid):
    v0 = time.perf_counter()

    # Extract the zip files
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    os.makedirs(flakyDir, exist_ok=True)
    os.makedirs(nonFlakyDir, exist_ok=True)
    
    extract_zip(flakyZip, flakyDir)
    extract_zip(nonFlakyZip, nonFlakyDir)

    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky

    print(f"Number of flaky documents: {len(dataPointsFlaky)}")
    print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
    print(f"Total number of documents: {len(dataPoints)}")
    
    if len(dataPoints) == 0:
        raise ValueError("No documents available for vectorization. Please check the input directories.")

    # Vectorization
    Z = flastVectorization(dataPoints, dim=dim, eps=eps)
    dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))
    vecTime = time.perf_counter() - v0

    # Define StratifiedKFold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Initialize storage for metrics per fold and threshold
    thresholds = np.linspace(0.1, 0.9, 9)
    metrics_per_combination = []

    # Manually iterate over all combinations of hyperparameters
    for params in product(*param_grid.values()):
        # Convert params from tuple to dictionary
        param_dict = dict(zip(param_grid.keys(), params))
        print(f"Training with parameters: {param_dict}")
        
        # Initialize Pipeline with PCA and KNN
        pipeline = Pipeline([
            ('pca', PCA(n_components=dim)),  # PCA step with dynamic components
            ('knn', KNeighborsClassifier(
                n_neighbors=param_dict['n_neighbors'],
                weights=param_dict['weights'],
                metric=param_dict['metric'],
                n_jobs=-1
            ))
        ])
        
        # Cross-validation
        for fold, (train_index, test_index) in enumerate(skf.split(Z, dataLabelsList)):
            X_train, X_test = Z[train_index], Z[test_index]
            y_train, y_test = dataLabelsList[train_index], dataLabelsList[test_index]

            if sum(y_train) == 0 or sum(y_test) == 0:
                print(f"Skipping fold {fold+1} due to no positive samples in train or test set")
                continue

            # Train the pipeline model (PCA + KNN)
            pipeline.fit(X_train, y_train)

            # Predict probabilities on test set
            if hasattr(pipeline.named_steps['knn'], "predict_proba"):
                y_pred_proba = pipeline.named_steps['knn'].predict_proba(X_test)
            else:
                distances, indices = pipeline.named_steps['knn'].kneighbors(X_test)
                weights = pipeline.named_steps['knn']._get_weights(distances)
                y_pred_proba = np.zeros((X_test.shape[0], 2))
                for i, neighbors in enumerate(indices):
                    neighbor_labels = y_train[neighbors]
                    if weights is None:
                        proba = np.bincount(neighbor_labels, minlength=2) / pipeline.named_steps['knn'].n_neighbors
                    else:
                        proba = np.bincount(neighbor_labels, weights=weights[i], minlength=2) / weights[i].sum()
                    y_pred_proba[i] = proba

            # Calculate metrics for each threshold
            for threshold in thresholds:
                y_pred = (y_pred_proba[:, 1] >= threshold).astype(int)

                # Calculate metrics for this threshold
                f1 = f1_score(y_test, y_pred, zero_division=1)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred, zero_division=1)
                recall = recall_score(y_test, y_pred, zero_division=1)
                mcc = matthews_corrcoef(y_test, y_pred)

                metrics_per_combination.append({
                    'n_neighbors': param_dict['n_neighbors'],
                    'weights': param_dict['weights'],
                    'metric': param_dict['metric'],
                    'fold': fold + 1,
                    'threshold': threshold,
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'mcc': mcc
                })

    if len(metrics_per_combination) == 0:
        print("No valid folds. Exiting.")
        return None, None

    # Convert the list of metrics into a DataFrame
    df_results = pd.DataFrame(metrics_per_combination)

    # Save the per-fold results
    outFile = f"{combination_label}-thresholds-knn-results-per-fold.csv"
    df_results.to_csv(os.path.join(outDir, outFile), index=False)

    print(f"Per-fold KNN analysis completed. Results saved to: {outFile}")

    # Compute the average metrics across folds for each combination of hyperparameters and thresholds
    grouped_metrics = df_results.groupby(
        ['n_neighbors', 'weights', 'metric', 'threshold']
    ).agg({
        'accuracy': ['mean', 'std'],
        'precision': ['mean', 'std'],
        'recall': ['mean', 'std'],
        'f1': ['mean', 'std'],
        'mcc': ['mean', 'std']
    }).reset_index()

    # Flatten the MultiIndex columns
    grouped_metrics.columns = [
        '_'.join(col).strip('_') if col[1] else col[0] 
        for col in grouped_metrics.columns.values
    ]

    # Save the averaged results
    outFileAvg = f"{combination_label}-thresholds-knn-results-averaged.csv"
    grouped_metrics.to_csv(os.path.join(outDir, outFileAvg), index=False)

    print(f"Averaged results saved to: {outFileAvg}")

    return df_results, grouped_metrics

if __name__ == "__main__":

    # Define the parameter grid for hyperparameter tuning
    param_grid = {
        'n_neighbors': [3, 5, 7, 9, 11, 15, 20],
        'weights': ['uniform', 'distance'],
        'metric': ['cosine', 'euclidean'],
    }

    # Paths to your datasets
    flakyZip = "compressedDataset/flaky_files.zip"
    # Use the same dataset for both models to ensure fairness
    # You can choose between 'reduced_nonflaky_files.zip' (balanced) or 'all_nonflaky_files.zip' (unbalanced)
    nonFlakyZip = "compressedDataset/all_nonflaky_files.zip"  # Unbalanced dataset
    # nonFlakyZip = "compressedDataset/reduced_nonflaky_files.zip"  # Balanced dataset

    # Create result and extract directories
    outDir = "results/knn_thresholds/"
    os.makedirs(outDir, exist_ok=True)
    extractDir = "extracted/knn_thresholds/"
    os.makedirs(extractDir, exist_ok=True)

    # Perform KNN analysis with threshold adjustments
    print("Starting KNN analysis with threshold adjustments...")
    df_results, grouped_metrics = flastKNNWithThresholds(
        outDir, flakyZip, nonFlakyZip, extractDir, n_splits=5, dim=100, eps=0.3,
        combination_label="knn_thresholds", param_grid=param_grid)

    if df_results is not None:
        print("Analysis completed. Per-fold Results:")
        print(df_results.head())

        print("Averaged Results:")
        print(grouped_metrics.head())
    else:
        print("Analysis did not produce any results.")


Starting KNN analysis with threshold adjustments...
Number of flaky documents: 45
Number of non-flaky documents: 254
Total number of documents: 299
Training with parameters: {'n_neighbors': 3, 'weights': 'uniform', 'metric': 'cosine'}
Training with parameters: {'n_neighbors': 3, 'weights': 'uniform', 'metric': 'euclidean'}
Training with parameters: {'n_neighbors': 3, 'weights': 'distance', 'metric': 'cosine'}
Training with parameters: {'n_neighbors': 3, 'weights': 'distance', 'metric': 'euclidean'}
Training with parameters: {'n_neighbors': 5, 'weights': 'uniform', 'metric': 'cosine'}
Training with parameters: {'n_neighbors': 5, 'weights': 'uniform', 'metric': 'euclidean'}
Training with parameters: {'n_neighbors': 5, 'weights': 'distance', 'metric': 'cosine'}
Training with parameters: {'n_neighbors': 5, 'weights': 'distance', 'metric': 'euclidean'}
Training with parameters: {'n_neighbors': 7, 'weights': 'uniform', 'metric': 'cosine'}
Training with parameters: {'n_neighbors': 7, 'weights

## XGB

In [8]:
import os
import time
import zipfile
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
)
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from itertools import product

###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList

def flastVectorization(dataPoints, dim=0, eps=0.3):
    """Performs vectorization using CountVectorizer with optional dimensionality reduction."""
    countVec = CountVectorizer(stop_words=None)  # No stop word removal
    Z_full = countVec.fit_transform(dataPoints)
    if eps == 0:
        Z = Z_full
    else:
        if dim <= 0:
            from sklearn.random_projection import johnson_lindenstrauss_min_dim
            dim = johnson_lindenstrauss_min_dim(Z_full.shape[0], eps=eps)
        from sklearn.random_projection import SparseRandomProjection
        srp = SparseRandomProjection(n_components=dim)
        Z = srp.fit_transform(Z_full)
    return Z

###############################################################################
# Modified SVM with Threshold Adjustment

def flastNBWithThresholds(outDir, flakyZip, nonFlakyZip, extractDir, n_splits, dim, eps, combination_label, param_grid):
    v0 = time.perf_counter()

    # Extract the zip files
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    os.makedirs(flakyDir, exist_ok=True)
    os.makedirs(nonFlakyDir, exist_ok=True)
    
    extract_zip(flakyZip, flakyDir)
    extract_zip(nonFlakyZip, nonFlakyDir)

    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky

    print(f"Number of flaky documents: {len(dataPointsFlaky)}")
    print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
    print(f"Total number of documents: {len(dataPoints)}")
    
    if len(dataPoints) == 0:
        raise ValueError("No documents available for vectorization. Please check the input directories.")

    # Vectorization
    Z = flastVectorization(dataPoints, dim=dim, eps=eps)
    dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))
    vecTime = time.perf_counter() - v0

    # Define StratifiedKFold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Initialize storage for metrics per fold and threshold
    thresholds = np.linspace(0.1, 0.9, 9)
    metrics_per_combination = []

    # Manually iterate over all combinations of hyperparameters
    for params in product(*param_grid.values()):
        # Convert params from tuple to dictionary
        param_dict = dict(zip(param_grid.keys(), params))
        print(f"Training with parameters: {param_dict}")
        
     # Initialize MultinomialNB with current hyperparameter combination
        nb_model = MultinomialNB(
            alpha=param_dict['alpha']
        )
        
        # Cross-validation
        for fold, (train_index, test_index) in enumerate(skf.split(Z, dataLabelsList)):
            X_train, X_test = Z[train_index], Z[test_index]
            y_train, y_test = dataLabelsList[train_index], dataLabelsList[test_index]

            if sum(y_train) == 0 or sum(y_test) == 0:
                print(f"Skipping fold {fold+1} due to no positive samples in train or test set")
                continue

            # Train the model
            svm_model.fit(X_train, y_train)

            # Predict probabilities on test set
            y_pred_proba = svm_model.predict_proba(X_test)

            # Calculate metrics for each threshold
            for threshold in thresholds:
                y_pred = (y_pred_proba[:, 1] >= threshold).astype(int)

                # Calculate metrics for this threshold
                f1 = f1_score(y_test, y_pred, zero_division=1)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred, zero_division=1)
                recall = recall_score(y_test, y_pred, zero_division=1)
                mcc = matthews_corrcoef(y_test, y_pred)

                metrics_per_combination.append({
                    'alpha': param_dict['alpha'],
                    'fold': fold + 1,
                    'threshold': threshold,
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'mcc': mcc
                })

    if len(metrics_per_combination) == 0:
        print("No valid folds. Exiting.")
        return None, None

    # Convert the list of metrics into a DataFrame
    df_results = pd.DataFrame(metrics_per_combination)

    # Save the per-fold results
    outFile = f"{combination_label}-thresholds-nb-results-per-fold.csv"
    df_results.to_csv(os.path.join(outDir, outFile), index=False)

    print(f"Per-fold SVM analysis completed. Results saved to: {outFile}")

    # Compute the average metrics across folds for each combination of hyperparameters and thresholds
    grouped_metrics = df_results.groupby(
        ['C', 'kernel', 'threshold']
    ).agg({
        'accuracy': ['mean', 'std'],
        'precision': ['mean', 'std'],
        'recall': ['mean', 'std'],
        'f1': ['mean', 'std'],
        'mcc': ['mean', 'std']
    }).reset_index()

    # Flatten the MultiIndex columns
    grouped_metrics.columns = [
        '_'.join(col).strip('_') if col[1] else col[0] 
        for col in grouped_metrics.columns.values
    ]

    # Save the averaged results
    outFileAvg = f"{combination_label}-thresholds-NB-result-averaged.csv"
    grouped_metrics.to_csv(os.path.join(outDir, outFileAvg), index=False)

    print(f"Averaged results saved to: {outFileAvg}")

    return df_results, grouped_metrics

if __name__ == "__main__":

    # Define the parameter grid for hyperparameter tuning
    param_grid = {
        'C': [0.01, 0.1, 1.0, 10.0, 100.0],  # Regularization parameter
        'kernel': ['linear', 'rbf', 'poly', 'sigmoid']  # Kernel types
    }

    # Paths to your datasets
    flakyZip = "compressedDataset/flaky_files.zip"
    # Use the same dataset for both models to ensure fairness
    # You can choose between 'reduced_nonflaky_files.zip' (balanced) or 'all_nonflaky_files.zip' (unbalanced)
    nonFlakyZip = "compressedDataset/all_nonflaky_files.zip"  # Unbalanced dataset
    # nonFlakyZip = "compressedDataset/reduced_nonflaky_files.zip"  # Balanced dataset

    # Create result and extract directories
    outDir = "results/NB_thresholds/"
    os.makedirs(outDir, exist_ok=True)
    extractDir = "extracted/NB_thresholds/"
    os.makedirs(extractDir, exist_ok=True)

    # Perform SVM analysis with threshold adjustments
    print("Starting NB analysis with threshold adjustments...")
    df_results, grouped_metrics = flastSVMWithThresholds(
        outDir, flakyZip, nonFlakyZip, extractDir, n_splits=5, dim=100, eps=0.3,
        combination_label="NB_thresholds", param_grid=param_grid)

    if df_results is not None:
        print("Analysis completed. Per-fold Results:")
        print(df_results.head())

        print("Averaged Results:")
        print(grouped_metrics.head())
    else:
        print("Analysis did not produce any results.")


Starting NB analysis with threshold adjustments...
Number of flaky documents: 45
Number of non-flaky documents: 254
Total number of documents: 299
Training with parameters: {'C': 0.01, 'kernel': 'linear'}
Training with parameters: {'C': 0.01, 'kernel': 'rbf'}
Training with parameters: {'C': 0.01, 'kernel': 'poly'}
Training with parameters: {'C': 0.01, 'kernel': 'sigmoid'}
Training with parameters: {'C': 0.1, 'kernel': 'linear'}
Training with parameters: {'C': 0.1, 'kernel': 'rbf'}
Training with parameters: {'C': 0.1, 'kernel': 'poly'}
Training with parameters: {'C': 0.1, 'kernel': 'sigmoid'}
Training with parameters: {'C': 1.0, 'kernel': 'linear'}
Training with parameters: {'C': 1.0, 'kernel': 'rbf'}
Training with parameters: {'C': 1.0, 'kernel': 'poly'}
Training with parameters: {'C': 1.0, 'kernel': 'sigmoid'}
Training with parameters: {'C': 10.0, 'kernel': 'linear'}
Training with parameters: {'C': 10.0, 'kernel': 'rbf'}
Training with parameters: {'C': 10.0, 'kernel': 'poly'}
Training

Starting KNN analysis with threshold adjustments...
Number of flaky documents: 45
Number of non-flaky documents: 254
Total number of documents: 299
Training with parameters: {'n_neighbors': 3, 'weights': 'uniform', 'metric': 'cosine'}
Training with parameters: {'n_neighbors': 3, 'weights': 'uniform', 'metric': 'euclidean'}
Training with parameters: {'n_neighbors': 3, 'weights': 'distance', 'metric': 'cosine'}
Training with parameters: {'n_neighbors': 3, 'weights': 'distance', 'metric': 'euclidean'}
Training with parameters: {'n_neighbors': 5, 'weights': 'uniform', 'metric': 'cosine'}
Training with parameters: {'n_neighbors': 5, 'weights': 'uniform', 'metric': 'euclidean'}
Training with parameters: {'n_neighbors': 5, 'weights': 'distance', 'metric': 'cosine'}
Training with parameters: {'n_neighbors': 5, 'weights': 'distance', 'metric': 'euclidean'}
Training with parameters: {'n_neighbors': 7, 'weights': 'uniform', 'metric': 'cosine'}
Training with parameters: {'n_neighbors': 7, 'weights

## Random Forest

In [15]:
import os
import time
import zipfile
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
)
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from itertools import product

###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList

def flastVectorization(dataPoints, dim=0, eps=0.3):
    """Performs vectorization using CountVectorizer with optional dimensionality reduction."""
    countVec = CountVectorizer(stop_words=None)  # No stop word removal
    Z_full = countVec.fit_transform(dataPoints)
    if eps == 0:
        Z = Z_full
    else:
        if dim <= 0:
            from sklearn.random_projection import johnson_lindenstrauss_min_dim
            dim = johnson_lindenstrauss_min_dim(Z_full.shape[0], eps=eps)
        from sklearn.random_projection import SparseRandomProjection
        srp = SparseRandomProjection(n_components=dim)
        Z = srp.fit_transform(Z_full)
    return Z

###############################################################################
# Modified RF with Threshold Adjustment

def flastRFWithThresholds(outDir, flakyZip, nonFlakyZip, extractDir, n_splits, dim, eps, combination_label, param_grid):
    v0 = time.perf_counter()

    # Extract the zip files
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    os.makedirs(flakyDir, exist_ok=True)
    os.makedirs(nonFlakyDir, exist_ok=True)
    
    extract_zip(flakyZip, flakyDir)
    extract_zip(nonFlakyZip, nonFlakyDir)

    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky

    print(f"Number of flaky documents: {len(dataPointsFlaky)}")
    print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
    print(f"Total number of documents: {len(dataPoints)}")
    
    if len(dataPoints) == 0:
        raise ValueError("No documents available for vectorization. Please check the input directories.")

    # Vectorization
    Z = flastVectorization(dataPoints, dim=dim, eps=eps)
    dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))
    vecTime = time.perf_counter() - v0

    # Define StratifiedKFold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Initialize storage for metrics per fold and threshold
    thresholds = np.linspace(0.1, 0.9, 9)
    metrics_per_combination = []

    # Manually iterate over all combinations of hyperparameters
    for params in product(*param_grid.values()):
        # Convert params from tuple to dictionary
        param_dict = dict(zip(param_grid.keys(), params))
        print(f"Training with parameters: {param_dict}")
        
        # Initialize KNeighborsClassifier with current hyperparameter combination
        rf_model = RandomForestClassifier(
            n_estimators=param_dict['n_estimators'],
            criterion=param_dict['criterion'],
            max_depth=param_dict['max_depth'],
            min_samples_split=param_dict['min_samples_split'],
            min_samples_leaf=param_dict['min_samples_leaf'],
            random_state=42,
            n_jobs=-1
        )

        # Cross-validation
        for fold, (train_index, test_index) in enumerate(skf.split(Z, dataLabelsList)):
            X_train, X_test = Z[train_index], Z[test_index]
            y_train, y_test = dataLabelsList[train_index], dataLabelsList[test_index]

            if sum(y_train) == 0 or sum(y_test) == 0:
                print(f"Skipping fold {fold+1} due to no positive samples in train or test set")
                continue

            # Train the model
            rf_model.fit(X_train, y_train)

            # Predict probabilities on test set
            if hasattr(rf_model, "predict_proba"):
                y_pred_proba = rf_model.predict_proba(X_test)
            else:
                # If predict_proba is not available, use distance-based probabilities
                distances, indices = rf_model.kneighbors(X_test)
                weights = rf_model._get_weights(distances)
                y_pred_proba = np.zeros((X_test.shape[0], 2))
                for i, neighbors in enumerate(indices):
                    neighbor_labels = y_train[neighbors]
                    if weights is None:
                        proba = np.bincount(neighbor_labels, minlength=2) / rf_model.n_neighbors
                    else:
                        proba = np.bincount(neighbor_labels, weights=weights[i], minlength=2) / weights[i].sum()
                    y_pred_proba[i] = proba

            # Calculate metrics for each threshold
            for threshold in thresholds:
                y_pred = (y_pred_proba[:, 1] >= threshold).astype(int)

                # Calculate metrics for this threshold
                f1 = f1_score(y_test, y_pred, zero_division=1)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred, zero_division=1)
                recall = recall_score(y_test, y_pred, zero_division=1)
                mcc = matthews_corrcoef(y_test, y_pred)

                metrics_per_combination.append({
                    'n_estimators': param_dict['n_estimators'],
                    'criterion': param_dict['criterion'],
                    'max_depth': param_dict['max_depth'],
                    'min_samples_split': param_dict['min_samples_split'],
                    'min_samples_leaf': param_dict['min_samples_leaf'],
                    'fold': fold + 1,
                    'threshold': threshold,
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'mcc': mcc
                })

    if len(metrics_per_combination) == 0:
        print("No valid folds. Exiting.")
        return None, None

    # Convert the list of metrics into a DataFrame
    df_results = pd.DataFrame(metrics_per_combination)

    # Save the per-fold results
    outFile = f"{combination_label}-params-rf-{n_splits}-folds-Threshold.csv"
    df_results.to_csv(os.path.join(outDir, outFile), index=False)

    print(f"Per-fold rf analysis completed. Results saved to: {outFile}")

    # Compute the average metrics across folds for each combination of hyperparameters and thresholds
    grouped_metrics = df_results.groupby(
    ['n_estimators', 'criterion', 'max_depth', 'min_samples_split', 'min_samples_leaf', 'threshold']
).agg({
    'accuracy': ['mean', 'std'],
    'precision': ['mean', 'std'],
    'recall': ['mean', 'std'],
    'f1': ['mean', 'std'],
    'mcc': ['mean', 'std']
}).reset_index()


    # Flatten the MultiIndex columns
    grouped_metrics.columns = [
        '_'.join(col).strip('_') if col[1] else col[0] 
        for col in grouped_metrics.columns.values
    ]

    # Save the averaged results
    outFileAvg = f"{combination_label}-thresholds-RF-results-averaged.csv"
    grouped_metrics.to_csv(os.path.join(outDir, outFileAvg), index=False)

    print(f"Averaged results saved to: {outFileAvg}")

    return df_results, grouped_metrics

if __name__ == "__main__":

    # Define the parameter grid for hyperparameter tuning
    param_grid = {
        'n_estimators': [10, 50, 100, 300, 500],  # Number of trees
        'max_depth': [10, 30, 50, 100, 300, 500],  # Maximum depth of each tree
        'min_samples_split': [2, 5],  # Minimum number of samples required to split an internal node
        'min_samples_leaf': [1, 2],  # Minimum number of samples required to be at a leaf node
        'criterion': ['gini', 'entropy'],  # Function to measure the quality of a split
    }


    # Paths to your datasets
    flakyZip = "compressedDataset/flaky_files.zip"
    nonFlakyZip = "compressedDataset/all_nonflaky_files.zip"  # Unbalanced dataset

    # Create result and extract directories
    outDir = "results/rf_thresholds/"
    os.makedirs(outDir, exist_ok=True)
    extractDir = "extracted/rf_thresholds/"
    os.makedirs(extractDir, exist_ok=True)

    # Perform rf analysis with threshold adjustments
    print("Starting rf analysis with threshold adjustments...")
    df_results, grouped_metrics = flastRFWithThresholds(
        outDir, flakyZip, nonFlakyZip, extractDir, n_splits=5, dim=100, eps=0.3,
        combination_label="rf_thresholds", param_grid=param_grid)

    if df_results is not None:
        print("Analysis completed. Per-fold Results:")
        print(df_results.head())

        print("Averaged Results:")
        print(grouped_metrics.head())
    else:
        print("Analysis did not produce any results.")


Starting rf analysis with threshold adjustments...
Number of flaky documents: 45
Number of non-flaky documents: 254
Total number of documents: 299
Training with parameters: {'n_estimators': 10, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 10, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 10, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 10, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 10, 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 10, 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 10, 'max_

Training with parameters: {'n_estimators': 50, 'max_depth': 30, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 50, 'max_depth': 30, 'min_samples_split': 5, 'min_samples_leaf': 2, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 50, 'max_depth': 30, 'min_samples_split': 5, 'min_samples_leaf': 2, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 50, 'max_depth': 50, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 50, 'max_depth': 50, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 50, 'max_depth': 50, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 50, 'max_depth': 50, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 50, 'max_depth': 50, 

Training with parameters: {'n_estimators': 100, 'max_depth': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 100, 'max_depth': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 100, 'max_depth': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 100, 'max_depth': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 100, 'max_depth': 100, 'min_samples_split': 5, 'min_samples_leaf': 2, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 100, 'max_depth': 100, 'min_samples_split': 5, 'min_samples_leaf': 2, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 100, 'max_depth': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 100, 'max_

Training with parameters: {'n_estimators': 300, 'max_depth': 300, 'min_samples_split': 5, 'min_samples_leaf': 2, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 300, 'max_depth': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 300, 'max_depth': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 300, 'max_depth': 500, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 300, 'max_depth': 500, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 300, 'max_depth': 500, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 300, 'max_depth': 500, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 300, 'm

## Decision Tree

In [20]:
import os
import time
import zipfile
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
)
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from itertools import product

###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList

def flastVectorization(dataPoints, dim=0, eps=0.3):
    """Performs vectorization using CountVectorizer with optional dimensionality reduction."""
    countVec = CountVectorizer(stop_words=None)  # No stop word removal
    Z_full = countVec.fit_transform(dataPoints)
    if eps == 0:
        Z = Z_full
    else:
        if dim <= 0:
            from sklearn.random_projection import johnson_lindenstrauss_min_dim
            dim = johnson_lindenstrauss_min_dim(Z_full.shape[0], eps=eps)
        from sklearn.random_projection import SparseRandomProjection
        srp = SparseRandomProjection(n_components=dim)
        Z = srp.fit_transform(Z_full)
    return Z

###############################################################################
# Modified ft with Threshold Adjustment

def flastdtWithThresholds(outDir, flakyZip, nonFlakyZip, extractDir, n_splits, dim, eps, combination_label, param_grid):
    v0 = time.perf_counter()

    # Extract the zip files
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    os.makedirs(flakyDir, exist_ok=True)
    os.makedirs(nonFlakyDir, exist_ok=True)
    
    extract_zip(flakyZip, flakyDir)
    extract_zip(nonFlakyZip, nonFlakyDir)

    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky

    print(f"Number of flaky documents: {len(dataPointsFlaky)}")
    print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
    print(f"Total number of documents: {len(dataPoints)}")
    
    if len(dataPoints) == 0:
        raise ValueError("No documents available for vectorization. Please check the input directories.")

    # Vectorization
    Z = flastVectorization(dataPoints, dim=dim, eps=eps)
    dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))
    vecTime = time.perf_counter() - v0

    # Define StratifiedKFold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Initialize storage for metrics per fold and threshold
    thresholds = np.linspace(0.1, 0.9, 9)
    metrics_per_combination = []

    # Manually iterate over all combinations of hyperparameters
    for params in product(*param_grid.values()):
        # Convert params from tuple to dictionary
        param_dict = dict(zip(param_grid.keys(), params))
        print(f"Training with parameters: {param_dict}")
        
        # Initialize KNeighborsClassifier with current hyperparameter combination
        dt_model = DecisionTreeClassifier(
            criterion=param_dict['criterion'],
            max_depth=param_dict['max_depth'],
            min_samples_split=param_dict['min_samples_split'],
            min_samples_leaf=param_dict['min_samples_leaf'],
            max_features=param_dict['max_features'],
            random_state=42
        )

        # Cross-validation
        for fold, (train_index, test_index) in enumerate(skf.split(Z, dataLabelsList)):
            X_train, X_test = Z[train_index], Z[test_index]
            y_train, y_test = dataLabelsList[train_index], dataLabelsList[test_index]

            if sum(y_train) == 0 or sum(y_test) == 0:
                print(f"Skipping fold {fold+1} due to no positive samples in train or test set")
                continue

            # Train the model
            dt_model.fit(X_train, y_train)

            # Predict probabilities on test set
            if hasattr(dt_model, "predict_proba"):
                y_pred_proba = dt_model.predict_proba(X_test)
            else:
                # If predict_proba is not available, use distance-based probabilities
                distances, indices = dt_model.kneighbors(X_test)
                weights = dt_model._get_weights(distances)
                y_pred_proba = np.zeros((X_test.shape[0], 2))
                for i, neighbors in enumerate(indices):
                    neighbor_labels = y_train[neighbors]
                    if weights is None:
                        proba = np.bincount(neighbor_labels, minlength=2) / dt_model.n_neighbors
                    else:
                        proba = np.bincount(neighbor_labels, weights=weights[i], minlength=2) / weights[i].sum()
                    y_pred_proba[i] = proba

            # Calculate metrics for each threshold
            for threshold in thresholds:
                y_pred = (y_pred_proba[:, 1] >= threshold).astype(int)

                # Calculate metrics for this threshold
                f1 = f1_score(y_test, y_pred, zero_division=1)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred, zero_division=1)
                recall = recall_score(y_test, y_pred, zero_division=1)
                mcc = matthews_corrcoef(y_test, y_pred)

                metrics_per_combination.append({
                    'criterion': param_dict['criterion'],
                    'max_depth': param_dict['max_depth'],
                    'min_samples_split': param_dict['min_samples_split'],
                    'min_samples_leaf': param_dict['min_samples_leaf'],
                    'max_features': param_dict['max_features'],
                    'fold': fold + 1,
                    'threshold': threshold,
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'mcc': mcc
                })

    if len(metrics_per_combination) == 0:
        print("No valid folds. Exiting.")
        return None, None

    # Convert the list of metrics into a DataFrame
    df_results = pd.DataFrame(metrics_per_combination)

    # Save the per-fold results
    outFile = f"{combination_label}-params-dt-{n_splits}-folds-Threshold.csv"
    df_results.to_csv(os.path.join(outDir, outFile), index=False)

    print(f"Per-fold dt analysis completed. Results saved to: {outFile}")

    # Compute the average metrics across folds for each combination of hyperparameters and thresholds
    grouped_metrics = df_results.groupby(
    ['criterion', 'max_depth', 'min_samples_split', 'min_samples_leaf', 'max_features', 'threshold']
).agg({
    'accuracy': ['mean', 'std'],
    'precision': ['mean', 'std'],
    'recall': ['mean', 'std'],
    'f1': ['mean', 'std'],
    'mcc': ['mean', 'std']
}).reset_index()

    # Flatten the MultiIndex columns
    grouped_metrics.columns = [
        '_'.join(col).strip('_') if col[1] else col[0] 
        for col in grouped_metrics.columns.values
    ]

    # Save the averaged results
    outFileAvg = f"{combination_label}-thresholds-dt-results-averaged.csv"
    grouped_metrics.to_csv(os.path.join(outDir, outFileAvg), index=False)

    print(f"Averaged results saved to: {outFileAvg}")

    return df_results, grouped_metrics

if __name__ == "__main__":

    # Define the parameter grid for hyperparameter tuning
    param_grid = {
    'criterion': ['gini', 'entropy'],  # Function to measure the quality of a split
    'max_depth': [None, 10, 30, 50, 100, 300, 500],  # Maximum depth of each tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 5, 10],  # Minimum number of samples required to be at a leaf node
    'max_features': [None, 'sqrt', 'log2'],  # Number of features to consider when looking for the best split
}



    # Paths to your datasets
    flakyZip = "compressedDataset/flaky_files.zip"
    nonFlakyZip = "compressedDataset/all_nonflaky_files.zip"  # Unbalanced dataset

    # Create result and extract directories
    outDir = "results/dt_thresholds/"
    os.makedirs(outDir, exist_ok=True)
    extractDir = "extracted/dt_thresholds/"
    os.makedirs(extractDir, exist_ok=True)

    # Perform rf analysis with threshold adjustments
    print("Starting dt analysis with threshold adjustments...")
    df_results, grouped_metrics = flastdtWithThresholds(
        outDir, flakyZip, nonFlakyZip, extractDir, n_splits=5, dim=100, eps=0.3,
        combination_label="dt_thresholds", param_grid=param_grid)

    if df_results is not None:
        print("Analysis completed. Per-fold Results:")
        print(df_results.head())

        print("Averaged Results:")
        print(grouped_metrics.head())
    else:
        print("Analysis did not produce any results.")


Starting dt analysis with threshold adjustments...
Number of flaky documents: 45
Number of non-flaky documents: 254
Total number of documents: 299
Training with parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None}
Training with parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2'}
Training with parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': None}
Training with parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'log2'}
Training with parameters: {'c

Training with parameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': None}
Training with parameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'log2'}
Training with parameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': None}
Training with parameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'log2'}
Training with parameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': None}
Training with parameters: {'criterion': 'gini',

Training with parameters: {'criterion': 'gini', 'max_depth': 50, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'log2'}
Training with parameters: {'criterion': 'gini', 'max_depth': 50, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': None}
Training with parameters: {'criterion': 'gini', 'max_depth': 50, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'gini', 'max_depth': 50, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'log2'}
Training with parameters: {'criterion': 'gini', 'max_depth': 50, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': None}
Training with parameters: {'criterion': 'gini', 'max_depth': 50, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'gini', 'max_depth': 50, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': 'log2'}
Training with parameters: {'criterion': 'gini', 'max

Training with parameters: {'criterion': 'gini', 'max_depth': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2'}
Training with parameters: {'criterion': 'gini', 'max_depth': 300, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': None}
Training with parameters: {'criterion': 'gini', 'max_depth': 300, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'gini', 'max_depth': 300, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'log2'}
Training with parameters: {'criterion': 'gini', 'max_depth': 300, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_features': None}
Training with parameters: {'criterion': 'gini', 'max_depth': 300, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'gini', 'max_depth': 300, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_features': 'log2'}
Training with parameters: {'criterion': 'gini

Training with parameters: {'criterion': 'gini', 'max_depth': 500, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': None}
Training with parameters: {'criterion': 'gini', 'max_depth': 500, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'gini', 'max_depth': 500, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'log2'}
Training with parameters: {'criterion': 'gini', 'max_depth': 500, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': None}
Training with parameters: {'criterion': 'gini', 'max_depth': 500, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'gini', 'max_depth': 500, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': 'log2'}
Training with parameters: {'criterion': 'gini', 'max_depth': 500, 'min_samples_split': 10, 'min_samples_leaf': 10, 'max_features': None}
Training with parameters: {'criterion':

Training with parameters: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'log2'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': None}
Training with parameters: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'log2'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': None}
Training with parameters: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': 'log2'}
Training with parameters: {'cri

Training with parameters: {'criterion': 'entropy', 'max_depth': 50, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 50, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': None}
Training with parameters: {'criterion': 'entropy', 'max_depth': 50, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 50, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'log2'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 50, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_features': None}
Training with parameters: {'criterion': 'entropy', 'max_depth': 50, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 50, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_features': 'log2'}
Training with parameters: {'cri

Training with parameters: {'criterion': 'entropy', 'max_depth': 100, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'log2'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 100, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': None}
Training with parameters: {'criterion': 'entropy', 'max_depth': 100, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 100, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'log2'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 100, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': None}
Training with parameters: {'criterion': 'entropy', 'max_depth': 100, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 100, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': 'log2'}
Training with par

Training with parameters: {'criterion': 'entropy', 'max_depth': 500, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'log2'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 500, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': None}
Training with parameters: {'criterion': 'entropy', 'max_depth': 500, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 500, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'log2'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 500, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': None}
Training with parameters: {'criterion': 'entropy', 'max_depth': 500, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 500, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': 'log2'}
Training with parameters

In [29]:
import os
import time
import zipfile
import math
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
)
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from itertools import product

###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList

###############################################################################
# PCA and KNN with Threshold Adjustment

def flastKNNWithThresholds(outDir, flakyZip, nonFlakyZip, extractDir, n_splits, combination_label, param_grid):
    v0 = time.perf_counter()

    # Extract the zip files
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    os.makedirs(flakyDir, exist_ok=True)
    os.makedirs(nonFlakyDir, exist_ok=True)
    
    extract_zip(flakyZip, flakyDir)
    extract_zip(nonFlakyZip, nonFlakyDir)

    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky

    print(f"Number of flaky documents: {len(dataPointsFlaky)}")
    print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
    print(f"Total number of documents: {len(dataPoints)}")
    
    if len(dataPoints) == 0:
        raise ValueError("No documents available for vectorization. Please check the input directories.")

    # Vectorization (PCA will be applied later in the pipeline)
    countVec = CountVectorizer(stop_words=None)
    Z_full = countVec.fit_transform(dataPoints).toarray()  # Convert to dense matrix

    dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))
    vecTime = time.perf_counter() - v0

    # Define StratifiedKFold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Initialize storage for metrics per fold and threshold
    thresholds = np.linspace(0.1, 0.9, 9)
    metrics_per_combination = []

    # Manually iterate over all combinations of hyperparameters
    for params in product(*param_grid.values()):
        # Convert params from tuple to dictionary
        param_dict = dict(zip(param_grid.keys(), params))
        print(f"Training with parameters: {param_dict}")

        # Pipeline setup: PCA followed by KNN
        pca = PCA(n_components=param_dict['pca__n_components'])
        knn_model = KNeighborsClassifier(
            n_neighbors=param_dict['knn__n_neighbors'],
            weights=param_dict['knn__weights'],
            metric=param_dict['knn__metric'],
            n_jobs=-1
        )
        pipeline = Pipeline([('pca', pca), ('knn', knn_model)])
        
        # Cross-validation
        for fold, (train_index, test_index) in enumerate(skf.split(Z_full, dataLabelsList)):
            X_train, X_test = Z_full[train_index], Z_full[test_index]
            y_train, y_test = dataLabelsList[train_index], dataLabelsList[test_index]

            if sum(y_train) == 0 or sum(y_test) == 0:
                print(f"Skipping fold {fold+1} due to no positive samples in train or test set")
                continue

            # Train the pipeline (PCA + KNN)
            pipeline.fit(X_train, y_train)

            # Predict probabilities on test set
            if hasattr(pipeline.named_steps['knn'], "predict_proba"):
                y_pred_proba = pipeline.predict_proba(X_test)
            else:
                # Handle the case if predict_proba is not available
                distances, indices = pipeline.named_steps['knn'].kneighbors(X_test)
                weights = pipeline.named_steps['knn']._get_weights(distances)
                y_pred_proba = np.zeros((X_test.shape[0], 2))
                for i, neighbors in enumerate(indices):
                    neighbor_labels = y_train[neighbors]
                    if weights is None:
                        proba = np.bincount(neighbor_labels, minlength=2) / pipeline.named_steps['knn'].n_neighbors
                    else:
                        proba = np.bincount(neighbor_labels, weights=weights[i], minlength=2) / weights[i].sum()
                    y_pred_proba[i] = proba

            # Calculate metrics for each threshold
            for threshold in thresholds:
                y_pred = (y_pred_proba[:, 1] >= threshold).astype(int)

                # Calculate metrics for this threshold
                f1 = f1_score(y_test, y_pred, zero_division=1)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred, zero_division=1)
                recall = recall_score(y_test, y_pred, zero_division=1)
                mcc = matthews_corrcoef(y_test, y_pred)

                metrics_per_combination.append({
                    'pca_n_components': param_dict['pca__n_components'],
                    'n_neighbors': param_dict['knn__n_neighbors'],
                    'weights': param_dict['knn__weights'],
                    'metric': param_dict['knn__metric'],
                    'fold': fold + 1,
                    'threshold': threshold,
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'mcc': mcc
                })

    if len(metrics_per_combination) == 0:
        print("No valid folds. Exiting.")
        return None, None

    # Convert the list of metrics into a DataFrame
    df_results = pd.DataFrame(metrics_per_combination)

    # Save the per-fold results
    outFile = f"{combination_label}-thresholds-knn-results-per-fold.csv"
    df_results.to_csv(os.path.join(outDir, outFile), index=False)

    print(f"Per-fold KNN analysis completed. Results saved to: {outFile}")

    # Compute the average metrics across folds for each combination of hyperparameters and thresholds
    grouped_metrics = df_results.groupby(
        ['pca_n_components', 'n_neighbors', 'weights', 'metric', 'threshold']
    ).agg({
        'accuracy': ['mean', 'std'],
        'precision': ['mean', 'std'],
        'recall': ['mean', 'std'],
        'f1': ['mean', 'std'],
        'mcc': ['mean', 'std']
    }).reset_index()

    # Flatten the MultiIndex columns
    grouped_metrics.columns = [
        '_'.join(col).strip('_') if col[1] else col[0] 
        for col in grouped_metrics.columns.values
    ]

    # Save the averaged results
    outFileAvg = f"{combination_label}-thresholds-knn-results-averaged.csv"
    grouped_metrics.to_csv(os.path.join(outDir, outFileAvg), index=False)

    print(f"Averaged results saved to: {outFileAvg}")

    return df_results, grouped_metrics

if __name__ == "__main__":

    # Define the parameter grid for hyperparameter tuning
    param_grid = {
        'pca__n_components': [math.floor(i*0.08*100) for i in range(7, 10)],  # Adjust based on dataset size
        'knn__n_neighbors': [3, 5, 7, 9, 11, 15, 20],
        'knn__metric': ['cosine', 'euclidean'],
        'knn__weights': ['uniform', 'distance'],
    }

    # Paths to your datasets
    flakyZip = "compressedDataset/flaky_files.zip"
    nonFlakyZip = "compressedDataset/all_nonflaky_files.zip"  # Unbalanced dataset

    # Create result and extract directories
    outDir = "results/knn_thresholds/"
    os.makedirs(outDir, exist_ok=True)
    extractDir = "extracted/knn_thresholds/"
    os.makedirs(extractDir, exist_ok=True)

    # Perform KNN analysis with threshold adjustments
    print("Starting KNN analysis with threshold adjustments...")
    df_results, grouped_metrics = flastKNNWithThresholds(
        outDir, flakyZip, nonFlakyZip, extractDir, n_splits=5,
        combination_label="knn_thresholds", param_grid=param_grid)

    if df_results is not None:
        print("Analysis completed. Per-fold Results:")
        print(df_results.head())

        print("Averaged Results:")
        print(grouped_metrics.head())
    else:
        print("Analysis did not produce any results.")


Starting KNN analysis with threshold adjustments...
Number of flaky documents: 45
Number of non-flaky documents: 254
Total number of documents: 299
Training with parameters: {'pca__n_components': 56, 'knn__n_neighbors': 3, 'knn__metric': 'cosine', 'knn__weights': 'uniform'}
Training with parameters: {'pca__n_components': 56, 'knn__n_neighbors': 3, 'knn__metric': 'cosine', 'knn__weights': 'distance'}
Training with parameters: {'pca__n_components': 56, 'knn__n_neighbors': 3, 'knn__metric': 'euclidean', 'knn__weights': 'uniform'}
Training with parameters: {'pca__n_components': 56, 'knn__n_neighbors': 3, 'knn__metric': 'euclidean', 'knn__weights': 'distance'}
Training with parameters: {'pca__n_components': 56, 'knn__n_neighbors': 5, 'knn__metric': 'cosine', 'knn__weights': 'uniform'}
Training with parameters: {'pca__n_components': 56, 'knn__n_neighbors': 5, 'knn__metric': 'cosine', 'knn__weights': 'distance'}
Training with parameters: {'pca__n_components': 56, 'knn__n_neighbors': 5, 'knn__

Training with parameters: {'pca__n_components': 72, 'knn__n_neighbors': 5, 'knn__metric': 'euclidean', 'knn__weights': 'distance'}
Training with parameters: {'pca__n_components': 72, 'knn__n_neighbors': 7, 'knn__metric': 'cosine', 'knn__weights': 'uniform'}
Training with parameters: {'pca__n_components': 72, 'knn__n_neighbors': 7, 'knn__metric': 'cosine', 'knn__weights': 'distance'}
Training with parameters: {'pca__n_components': 72, 'knn__n_neighbors': 7, 'knn__metric': 'euclidean', 'knn__weights': 'uniform'}
Training with parameters: {'pca__n_components': 72, 'knn__n_neighbors': 7, 'knn__metric': 'euclidean', 'knn__weights': 'distance'}
Training with parameters: {'pca__n_components': 72, 'knn__n_neighbors': 9, 'knn__metric': 'cosine', 'knn__weights': 'uniform'}
Training with parameters: {'pca__n_components': 72, 'knn__n_neighbors': 9, 'knn__metric': 'cosine', 'knn__weights': 'distance'}
Training with parameters: {'pca__n_components': 72, 'knn__n_neighbors': 9, 'knn__metric': 'euclide